# Data and Impressions

In [25]:
from db_config.db_setup import Site, Address, engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy import select
from IPython.display import display, HTML

Session = sessionmaker(bind=engine)
session = Session()

query = session.query(Site, Address).join(Address, Site.id == Address.domain_id)

with engine.connect() as connection:
    result = connection.execute(query.statement)
    
df = pd.DataFrame(result.fetchall(), columns=result.keys())
df.drop(columns=['id_1', 'domain_id', 'region'], inplace=True)
df.drop_duplicates(inplace=True)

display(HTML(df.to_html()))
session.close()


2023-06-11 20:25:16,870 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-11 20:25:16,871 INFO sqlalchemy.engine.Engine SELECT site.id, site.domain, address.id AS id_1, address.domain_id, address.country, address.region, address.state, address.city, address.postcode, address.road, address.road_number 
FROM site JOIN address ON site.id = address.domain_id
2023-06-11 20:25:16,872 INFO sqlalchemy.engine.Engine [cached since 4718s ago] ()
2023-06-11 20:25:16,873 INFO sqlalchemy.engine.Engine ROLLBACK


,id,domain,country,state,city,postcode,road,road_number
0,1,umbrawindowtinting.com,None,None,None,None,bloomingdale rd.,None
2,1,umbrawindowtinting.com,None,None,None,None,w higgins rd # b,None
4,1,umbrawindowtinting.com,None,None,bensenville,None,None,None
5,1,umbrawindowtinting.com,None,None,wheaton,None,wood dale,None
6,1,umbrawindowtinting.com,None,il,glendale heights,60139,None,None
8,1,umbrawindowtinting.com,None,il,glendale heights,60139,bloomingdale rd,None
10,1,umbrawindowtinting.com,None,il,schaumburg,60195,None,None
12,2,embcmonroe.org,None,nc,monroe,28112,maurice street,None
13,11,saintmlc.com,None,None,None,None,e.maine st.,None
15,15,wyandottewinery.com,None,ohio,columbus,43230,None,None


## Why

    - SQLAlchemy + SQLite. I think this combo works very well, as it combines the flexibility of SQLAlchemy in dealing with diverse situations while mantaining the readability of the code with the portable and integrable nature of SQLite. As a proof, in the cell above I simply imported my database, joined two tables and sent the data to pandas to create a scrollable output. As for pandas, it's just naturaly to use it when it has such versatility 
    - Scrapy is a powerful tool for web crawling and one aspect that proved very useful is its customizability. There are many settings to tweak in order to increase the performance and the reliance of scraping, many ways to create a good spider and to adapt it to the target site. In my case, I even relied on bs4 to proccess the retrieved data and had no problem to integrate it with my spider. Also, pipelines are definetly great for sending data
    - transformers. Its large number of models makes it quite intersting to use. Tried other models, like spacy, but transformers worked the best for the task. At first I wanted to use Babelscape/wikineural-multilingual-ner, but BERT was a bit more reliable
    - as for pypostal, it has its niche and works well. But, the data definetly requires a bit of preprocessing before

## Personal thoughts on the project:

    - Extracting addresses proved harder than expected, especially when you want to do it strictly using NLP, without regex or other stuff. Since there were some sites in other languages I tried to solve the problem as universal as I could
    - Found using pretrained models quite inefficient and bothersome. Tried Babelscape/wikineural-multilingual-ner, dslim/bert-base-NER, SpaCy models, nltk models and each one had their own set of problems when trying to locate addresses on a page
    - too many 'state of art ML models' that would categorize phone numbers as addresses (an example)
    - Some sites are really bad written and unstructured. Some would look good, but have information all over the place. Some may not give any repsone at all. Other would detect Scrapy, had no chance. I need be honest though, was quite intersting to look through them
    - It takes quite a bit to crawl through all the domains, I scraped only pages like home, contact and location to save time
    - Trying to model a database is fun
    - While the project could have turned better, I learned a lot doing it. At this moment, I'm pleased with the final result and so more with what I learned

## How it can be improved:

    - custom model for extracting the addresses
    - function to remove the duplicates from the result
    - better international support
    - I left the program to process all home, location, contact pages, custom logic can be implemented to decide what pages to scrape
    - not so good support if multiple addresses are present
    - in some cases addresses can be all over the place in the database, better cleaning function
    - add column 'success' in the table with domains stating if the operation was successful
    - in case the sites are multiple times scraped (because of errors, improved models/spider) add column for time, version
    - adding proxies to bypass scraping detection